<a href="https://colab.research.google.com/github/nogbazghi/CSC8980/blob/main/HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nahom Ogbazghi
002052292

In [47]:
!pip install wget

In [48]:
!mkdir -p data
!wget -nc http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip -P data
!unzip -n -d data data/trainingandtestdata.zip

URL transformed to HTTPS due to an HSTS policy
File ‘data/trainingandtestdata.zip’ already there; not retrieving.

Archive:  data/trainingandtestdata.zip


In [49]:
import pandas as pd
# Supress deprecation warnings
import logging
logging.getLogger('tensorflow').disabled = True

tweets = pd.read_csv("./data/testdata.manual.2009.06.14.csv", delimiter=',', header=None, names=['polarity', 'id', 'date', 'query', 'user', 'text'])

# Map for readable classnames
class_names = ["Negative", "Positive"]

# removed neutral
filteredTweets = tweets[tweets.polarity != 2]
#relabeled positive from 4 to 1
filteredTweets['polarity'].replace({4: 1}, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [50]:
filteredTweets.shape
filteredTweets.polarity.value_counts()

1    182
0    177
Name: polarity, dtype: int64

1. Take the positive and the negative tweets only. Use Sklearn to split the dataset in 80%
training, 20% testing splits. Provide a nicely formatted summary of these splits,
containing their size) (15 points)

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [52]:
vocab_size=10000
vectorizer = TfidfVectorizer(max_features=vocab_size)
vectors = vectorizer.fit_transform(filteredTweets.text)
word_id_pair = vectorizer.vocabulary_
words_df = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names())
# words_df.head()

In [53]:
X = words_df
y = filteredTweets.polarity

In [54]:
from sklearn.model_selection import train_test_split

randomSeed = 2361

# a 80% training, 20 test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=randomSeed)

In [55]:
from tabulate import tabulate

training_indexes = y_train.index.values.tolist()
testing_indexes = y_test.index.values.tolist()
training_summary = ""
testing_summary = ""
training_positive = 0
testing_positive = 0
for index, row in filteredTweets.iterrows():
  if index in training_indexes:
    training_summary += row.text + " "
    if row.polarity == 1:
      training_positive += 1
  else:
    testing_summary += row.text + " "
    if row.polarity == 1:
      testing_positive += 1

training_data = ["Training", training_summary[:65], len(training_indexes), training_positive, len(training_indexes) - training_positive]
testing_data = ["Testing", testing_summary[:65], len(testing_indexes), testing_positive, len(testing_indexes) - testing_positive]
data = [training_data, testing_data]

print(tabulate(data, headers=["Data Type", "Text Summary", "Amount of Items", "Positive Count", "Negative Count"], tablefmt="fancy_grid"))

╒═════════════╤══════════════════════════════════════════════════════════════════╤═══════════════════╤══════════════════╤══════════════════╕
│ Data Type   │ Text Summary                                                     │   Amount of Items │   Positive Count │   Negative Count │
╞═════════════╪══════════════════════════════════════════════════════════════════╪═══════════════════╪══════════════════╪══════════════════╡
│ Training    │ @stellargirl I loooooooovvvvvveee my Kindle2. Not that the DX is │               287 │              140 │              147 │
├─────────────┼──────────────────────────────────────────────────────────────────┼───────────────────┼──────────────────┼──────────────────┤
│ Testing     │ Ok, first assesment of the #kindle2 ...it fucking rocks!!! Loves │                72 │               42 │               30 │
╘═════════════╧══════════════════════════════════════════════════════════════════╧═══════════════════╧══════════════════╧══════════════════╛


2. Use the code from the previous classes to build the following models (15 points):
a) SVM using TF-IDF.
b) Naive Bayes using TF-IDF.
c) Random Forest using TF-IDF.


In [56]:
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

In [57]:
print("Train A SVM")
# c
# Create and train a linear support vector classifier (LinearSVC)
SVM = LinearSVC(random_state=randomSeed)
SVM.fit(X_train, y_train)

Train A SVM


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=2361, tol=0.0001,
          verbose=0)

In [58]:
print("Train B NB")
# b
# Create and train a multinomial naive bayes classifier (MultinomialNB)
NB = MultinomialNB()
NB.fit(X_train, y_train)

Train B NB


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [59]:
print("Train C RF")
# c
# Create and train a random forest classifier
forest = RandomForestClassifier(random_state=randomSeed)
forest.fit(X_train, y_train)

Train C RF


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=2361,
                       verbose=0, warm_start=False)

In [60]:
import sklearn.metrics
from sklearn.metrics import confusion_matrix

# A
svm_true = y_test
svm_labels = SVM.predict(X_test)

In [61]:
# B
NB_true = y_test
NB_labels = NB.predict(X_test)

In [62]:
# C
forest_true = y_test
forest_labels = forest.predict(X_test)

3. Use the code from the LSTM class to build a classifier for negative and positive
sentiment tweets. Train the model with the training data split. Once the model is built,
test it with the testing data split. Display the classifier report for this evaluation. Answer
the following question: What can you say about the performance of this model? (40
points)


In [63]:
# All the imports!
import tensorflow as tf 
import numpy as np
from tensorflow.keras.preprocessing import sequence
from numpy import array
import re
import random

tf.random.set_seed(randomSeed)
random.seed(randomSeed)
np.random.seed(randomSeed)

In [64]:
# need to extract words from difficult text
getWord = "[0-9.!:(@)?\\-\*\#]*([a-zA-Z]+)[0-9.(@:)!?\\-\*\#]*"
emojis = ["(:", "[:", "]:", "):", "/:", "|:",":(", ":[", ":]", ":)", ":/", ":|"]
compile = re.compile(getWord)
urlStarter = "http"

#convert text to ints
intTweets = []
for tweet in filteredTweets.text:
  split = tweet.split(" ")
  wordInt = []
  for raw_word in split:
    word = raw_word.lower()
    if len(word) > 2 and not word.startswith(urlStarter):
        # check if emoji is in word
        # if any(emoji in word for emoji in emojis):
        for emoji in emojis:
          if(emoji in word):
            split.append(emoji)
    if word not in word_id_pair:
      match = compile.match(word)
      if match:
        grabbedWord = match.group(1)
        if grabbedWord in word_id_pair:
          wordInt.append(word_id_pair[grabbedWord])
        else:
          randomValForWord = random.random()
          word_id_pair[word] = randomValForWord
          wordInt.append(randomValForWord)
    else:
      wordInt.append(word_id_pair[word])
  intTweets.append(wordInt)

In [65]:
#need to provide uniform length
#since mean is 83 and max is 144 we should raise the average to the mid point (110)
review_length = 110

# Padding / truncated our reviews
intTweets_padded = sequence.pad_sequences(intTweets, maxlen = review_length)

# split the tweet using the same randomseed
# a 80% training, 20 test
tweets_train, tweets_test, y_train, y_test = train_test_split(intTweets_padded, y, train_size=0.8, random_state=randomSeed)

In [66]:
# We begin by defining the a empty stack. We'll use this for building our 
# network, later by layer.
model = tf.keras.models.Sequential()

# The Embedding Layer provides a spatial mapping (or Word Embedding) of all the 
# individual words in our training set. Words close to one another share context 
# and or meaning. This spatial mapping is learning during the training process.
model.add(
    tf.keras.layers.Embedding(
        input_dim = vocab_size, # The size of our vocabulary 
        output_dim = 32, # Dimensions to which each words shall be mapped
        input_length = review_length # Length of input sequences
    )
)

# Dropout layers fight overfitting and forces the model to learn multiple 
# representations of the same data by randomly disabling neurons in the 
# learning phase.
model.add(
    tf.keras.layers.Dropout(
        rate=0.25 # Randomly disable 25% of neurons
    )
)

# We are using a fast version of LSTM whih is optimised for GPUs. This layer 
# looks at the sequence of words in the review, along with their word embeddings
# and uses both of these to determine to sentiment of a given review.
model.add(
    tf.keras.layers.LSTM(
        units=32 # 32 LSTM units in this layer
    )
)

# Add a second dropout layer with the same aim as the first.
model.add(
    tf.keras.layers.Dropout(
        rate=0.25 # Randomly disable 25% of neurons
    )
)

# All LSTM units are connected to a single node in the dense layer. A sigmoid 
# activation function determines the output from this node - a value 
# between 0 and 1. Closer to 0 indicates a negative review. Closer to 1 
# indicates a positive review.
model.add(
    tf.keras.layers.Dense(
        units=1, # Single unit
        activation='sigmoid' # Sigmoid activation function (output from 0 to 1)
    )
)

# Compile the model
model.compile(
    loss=tf.keras.losses.binary_crossentropy, # loss function
    optimizer=tf.keras.optimizers.Adam(), # optimiser function
    metrics=['accuracy']) # reporting metric

# Display a summary of the models structure
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 110, 32)           320000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 110, 32)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 328,353
Trainable params: 328,353
Non-trainable params: 0
_________________________________________________________________


In [67]:
# Train the LSTM on the training data
history = model.fit(

    # Training data : features (review) and classes (positive or negative)
    tweets_train, y_train,
                    
    # Number of samples to work through before updating the 
    # internal model parameters via back propagation. The 
    # higher the batch, the more memory you need.
    batch_size=128, 

    # An epoch is an iteration over the entire training data.
    epochs=3, 
    
    # The model will set apart his fraction of the training 
    # data, will not train on it, and will evaluate the loss
    # and any model metrics on this data at the end of 
    # each epoch.
    validation_split=0.2,
    
    verbose=1
) 

Epoch 1/3
2/2 [==============================] - 3s 536ms/step - loss: 0.6938 - accuracy: 0.4399 - val_loss: 0.6931 - val_accuracy: 0.4138
Epoch 2/3
2/2 [==============================] - 0s 52ms/step - loss: 0.6922 - accuracy: 0.5630 - val_loss: 0.6929 - val_accuracy: 0.4655
Epoch 3/3
2/2 [==============================] - 0s 54ms/step - loss: 0.6909 - accuracy: 0.5471 - val_loss: 0.6928 - val_accuracy: 0.4655


In [68]:
# Get Model Predictions for test data
from sklearn.metrics import classification_report
predicted_classes = model.predict_classes(tweets_test)
classRep = (classification_report(y_test, predicted_classes, target_names=class_names))
print(classRep)
reportArray = (classRep.split())
precision = (float(reportArray[19])*100.0)
recall = (float(reportArray[20])*100.0)
fOne = (float(reportArray[21])*100.0)
lsmt_scores = ["LSMT", precision, recall, fOne]

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


              precision    recall  f1-score   support

    Negative       0.42      1.00      0.59        30
    Positive       0.00      0.00      0.00        42

    accuracy                           0.42        72
   macro avg       0.21      0.50      0.29        72
weighted avg       0.17      0.42      0.25        72



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**ANSWER:**

The model did not perform well at all. My assumption is that the corpus/tweets were not tokenized well enough, and there was too much noise in the data to calculate an accurate assumption on what the tweet was.

4. Compare all models together in terms of Precision, Recall and F1 score. Put all of
these numbers in a nicely formatted dataframe. Answer the following questions: Which
model performs the best? Why do you think this is? What do you think you can do to
improve performance? (30 points)


**ANSWER**

The model perferfomed fairly wild based on Negative and Positive. Negative and Postive scored a 100% for recall and precision respectively but were at 50% or lower for precison and recall respectively. The F1-score was not good either at 68% and 50%. 
The averages were more consistent but still not as good as expected, precision was over 75%, recall hovered around 60-70% and f1-score was about 60%. I do not think the model performed well but I believe that is directly associated to the quality of the data, much of the words were mispelled, filled with slang, and required strong context. A better parser could have been used to filter the text into numbers.

In [69]:
def scores(name, testData, labels):
  return [name, sklearn.metrics.precision_score(testData, labels)*100.0, sklearn.metrics.recall_score(testData, labels)*100.0, sklearn.metrics.f1_score(labels, testData, average='macro')*100.0]

In [70]:
# forest scores
forest_scores = scores("Forest", forest_true, forest_labels)

In [71]:
#naive baiez scores
nb_scores = scores("NB", NB_true, NB_labels)

In [72]:
# svm scores
svm_scores = scores("SVM", svm_true, svm_labels)

In [73]:
data = [svm_scores, nb_scores, forest_scores, lsmt_scores]
columns = ["Name", "Precision", "Recall", "F1 Measure"]
scores = pd.DataFrame(data=data,columns=columns)
print("Scores")
print(tabulate(scores, headers='keys', tablefmt='fancy_grid'))

Scores
╒════╤════════╤═════════════╤══════════╤══════════════╕
│    │ Name   │   Precision │   Recall │   F1 Measure │
╞════╪════════╪═════════════╪══════════╪══════════════╡
│  0 │ SVM    │     80.5556 │  69.0476 │      72.028  │
├────┼────────┼─────────────┼──────────┼──────────────┤
│  1 │ NB     │     85.7143 │  71.4286 │      76.2745 │
├────┼────────┼─────────────┼──────────┼──────────────┤
│  2 │ Forest │     79.4118 │  64.2857 │      69.3498 │
├────┼────────┼─────────────┼──────────┼──────────────┤
│  3 │ LSMT   │     21      │  50      │      29      │
╘════╧════════╧═════════════╧══════════╧══════════════╛


**ANSWER**:

Naive Bayes performed the best, with SVM closely behind I belive that is because the dataset provided was fairly small (under 400 items) and both of these algorithms work well with small sets. I believe Naive performed the best because tweets generally do not have a wide vocabularly and similar words are used repetitively which works in Naive Bayes favor. 
What can be done to improve the scores is increase the amount of tweets, the variety of tweets, and to better tokenize the tweets to feed the algorithms.

5. Add to the comparison of #4 a the manually calculated precision, recall and F1 score
using VADER and their suggested defaults to categorize the test split tweets in positive
or negative. Answer the following questions: Is this approach as good as the previous
ones? Why do you think this is? (30 points)

In [74]:
import nltk
nltk.download('vader_lexicon')  

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [75]:
def getPrecision(true_p, false_p):
  denom = (true_p + false_p) * 1.0
  return (true_p/denom) * 100

def getRecall(true_p, false_n):
  denom = (true_p + false_n) * 1.0
  return (true_p/denom) * 100

def getfOne(true_p, false_p, false_n):
  prec = getPrecision(true_p, false_p)
  recall = getRecall(true_p, false_n)
  num = 2.0 * prec * recall
  denom = (prec + recall)
  return (num/denom)

In [76]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

sia = SIA()

true_positive = 0
true_negative = 0
false_positive = 0
false_negative = 0
print("Test Tweets")
for index, row in filteredTweets.iterrows():
  className = class_names[row.polarity]
  tweet = (row.text)
  score = sia.polarity_scores(tweet)
  compound = float(score['compound'])
  neutral = float(score['neu'])
  negative = float(score['neg'])
  positive = float(score['pos'])

  if index in training_indexes:   
    if negative == positive:
      split = tweet.split()
    if className == "Positive":
      if positive > negative:
        true_positive += 1
      else:
        false_negative += 1
    else:
      if negative > positive:
        true_negative += 1
      else:
        false_positive += 1
  if index in testing_indexes:
    print("Tweet:",tweet)
    print("Actual:",className)
    if positive > negative:
      print("Predicted: Positive")
    elif positive < negative:
      print("Predicted: Negative")
    else:
      print("Predicted: Neutral")
    print()

precision = getPrecision(true_positive, false_positive) 
recall = getRecall(true_positive, false_negative)
fOne = getfOne(true_positive, false_positive, false_negative)


print("SCORES")
vader_scores = ["VADER", precision, recall, fOne]
data=[vader_scores]
print(tabulate(data, headers=columns, tablefmt="fancy_grid"))

Test Tweets
Tweet: Ok, first assesment of the #kindle2 ...it fucking rocks!!!
Actual: Positive
Predicted: Positive

Tweet: Loves twitter
Actual: Positive
Predicted: Positive

Tweet: how can you not love Obama? he makes jokes about himself.
Actual: Positive
Predicted: Negative

Tweet: House Correspondents dinner was last night whoopi, barbara &amp; sherri went, Obama got a standing ovation
Actual: Positive
Predicted: Neutral

Tweet: Watchin Espn..Jus seen this new Nike Commerical with a Puppet Lebron..sh*t was hilarious...LMAO!!!
Actual: Positive
Predicted: Neutral

Tweet: #lebron best athlete of our generation, if not all time (basketball related) I don't want to get into inter-sport debates about   __1/2
Actual: Positive
Predicted: Positive

Tweet: @ludajuice Lebron is a Beast, but I'm still cheering 4 the A..til the end.
Actual: Negative
Predicted: Positive

Tweet: @Pmillzz lebron IS THE BOSS
Actual: Positive
Predicted: Neutral

Tweet: good news, just had a call from the Visa office,

**ANSWER:**

This approach is as good as the others because the lexicon takes into consideration the context of tweets in the corpus used. So it considers items like text emojis when calculating the score.

Bonus (30 points): Try the following things to improve the LSTM model:
1) Use 90% training data, 10% testing
2) Remove stopwords from the tweets.
3) Remove all user mentions for the tweets (@something)
Compare all three new models in terms of their precision, recall and F1 score. Answer the
following questions: Did this change the results in any way? Why do you think so?

Split Text 90/10 Training/Test

In [77]:
#imports
from sklearn.metrics import classification_report
import os
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
import re
import random


In [78]:
#need to provide uniform length
#since mean is 83 and max is 144 we should raise the average to the mid point (110)
review_length = 110

intTweets_padded = sequence.pad_sequences(intTweets, maxlen = review_length)

# split the tweet using the same randomseed
#a 90% training, 10 test
a_tweets_train, a_tweets_test, a_y_train, a_y_test = train_test_split(intTweets_padded, y, train_size=0.9, random_state=randomSeed)

In [79]:
# Train the LSTM on the training data
history = model.fit(

    # Training data : features (review) and classes (positive or negative)
    a_tweets_train, a_y_train,
                    
    # Number of samples to work through before updating the 
    # internal model parameters via back propagation. The 
    # higher the batch, the more memory you need.
    batch_size=128, 

    # An epoch is an iteration over the entire training data.
    epochs=3, 
    
    # The model will set apart his fraction of the training 
    # data, will not train on it, and will evaluate the loss
    # and any model metrics on this data at the end of 
    # each epoch.
    validation_split=0.2,
    
    verbose=1
) 

Epoch 1/3
3/3 [==============================] - 0s 57ms/step - loss: 0.6893 - accuracy: 0.5620 - val_loss: 0.6932 - val_accuracy: 0.4462
Epoch 2/3
3/3 [==============================] - 0s 31ms/step - loss: 0.6879 - accuracy: 0.5271 - val_loss: 0.6928 - val_accuracy: 0.4462
Epoch 3/3
3/3 [==============================] - 0s 32ms/step - loss: 0.6851 - accuracy: 0.5775 - val_loss: 0.6919 - val_accuracy: 0.4462


In [80]:
# Get Model Predictions for test data
predicted_classes = model.predict_classes(a_tweets_test)
classRep = (classification_report(a_y_test, predicted_classes, target_names=class_names))
reportArray = (classRep.split())
precision = (float(reportArray[19])*100.0)
recall = (float(reportArray[20])*100.0)
fOne = (float(reportArray[21])*100.0)
nine_one_split = ["90/10 Split", precision, recall, fOne]

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Remove StopWords

In [81]:
nltk.download('stopwords')
nltk.download('punkt')

#get stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [82]:
random.seed(randomSeed)
# need to extract words from difficult text
getWord = "[0-9.!:(@)?\\-\*\#]*([a-zA-Z]+)[0-9.(@:)!?\\-\*\#]*"
emojis = ["(:", "[:", "]:", "):", "/:", "|:",":(", ":[", ":]", ":)", ":/", ":|"]
compile = re.compile(getWord)
urlStarter = "http"
#convert text to ints
b_intTweets = []
for tweet in filteredTweets.text:
  split = tweet.split(" ")
  wordInt = []
  for raw_word in split:
    word = raw_word.lower()
    if word not in stop_words:
      if len(word) > 2 and not word.startswith(urlStarter):
        # check if emoji is in word
        # if any(emoji in word for emoji in emojis):
        for emoji in emojis:
          if(emoji in word):
            split.append(emoji)
      if word not in word_id_pair:
        match = compile.match(word)
        if match:
          grabbedWord = match.group(1)
          if grabbedWord in word_id_pair:
            wordInt.append(word_id_pair[grabbedWord])
          else:
            randomValForWord = random.random()
            word_id_pair[word] = randomValForWord
            wordInt.append(randomValForWord)
      else:
        wordInt.append(word_id_pair[word])
  b_intTweets.append(wordInt)

In [83]:
# Review lengths across test and training whole datasets
print("Maximum review length: {}".format(len(max((b_intTweets), key=len))))
print("Minimum review length: {}".format(len(min((b_intTweets), key=len))))
result = [len(x) for x in b_intTweets]
print("Mean review length: {}".format(np.mean(result)))

#need to provide uniform length
#since mean is 9 and max is 20 we should raise the average to the mid point (14)
review_length = 14

# Padding / truncated our reviews
b_intTweets_padded = sequence.pad_sequences(b_intTweets, maxlen = review_length)

# split the tweet using the same randomseed
# a 80% training, 20 test
b_tweets_train, b_tweets_test, b_y_train, b_y_test = train_test_split(b_intTweets_padded, y, train_size=0.8, random_state=randomSeed)

Maximum review length: 20
Minimum review length: 2
Mean review length: 9.002785515320335


In [84]:
# Train the LSTM on the training data
history = model.fit(

    # Training data : features (review) and classes (positive or negative)
    b_tweets_train, b_y_train,
                    
    # Number of samples to work through before updating the 
    # internal model parameters via back propagation. The 
    # higher the batch, the more memory you need.
    batch_size=128, 

    # An epoch is an iteration over the entire training data.
    epochs=3, 
    
    # The model will set apart his fraction of the training 
    # data, will not train on it, and will evaluate the loss
    # and any model metrics on this data at the end of 
    # each epoch.
    validation_split=0.2,
    
    verbose=1
) 

Epoch 1/3
2/2 [==============================] - 2s 531ms/step - loss: 0.6812 - accuracy: 0.7031 - val_loss: 0.6908 - val_accuracy: 0.4828
Epoch 2/3
2/2 [==============================] - 0s 47ms/step - loss: 0.6804 - accuracy: 0.7380 - val_loss: 0.6902 - val_accuracy: 0.5172
Epoch 3/3
2/2 [==============================] - 0s 47ms/step - loss: 0.6779 - accuracy: 0.8166 - val_loss: 0.6896 - val_accuracy: 0.5345


In [85]:
# Get Model Predictions for test data
from sklearn.metrics import classification_report
predicted_classes = model.predict_classes(b_tweets_test)
classRep = (classification_report(b_y_test, predicted_classes, target_names=class_names))
reportArray = (classRep.split())
precision = (float(reportArray[19])*100.0)
recall = (float(reportArray[20])*100.0)
fOne = (float(reportArray[21])*100.0)
no_stop_words_scores = ["No Stop Words", precision, recall, fOne]

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Remove @words

In [86]:
random.seed(randomSeed)
# need to extract words from difficult text
getWord = "[0-9.!:(@)?\\-\*\#]*([a-zA-Z]+)[0-9.(@:)!?\\-\*\#]*"
emojis = ["(:", "[:", "]:", "):", "/:", "|:",":(", ":[", ":]", ":)", ":/", ":|"]
compile = re.compile(getWord)
urlStarter = "http"
#convert text to ints
c_intTweets = []
for tweet in filteredTweets.text:
  split = tweet.split(" ")
  wordInt = []
  for raw_word in split:
    if not raw_word.startswith("@"):
      word = raw_word.lower()
      if len(word) > 2 and not word.startswith(urlStarter):
          # check if emoji is in word
          # if any(emoji in word for emoji in emojis):
          for emoji in emojis:
            if(emoji in word):
              split.append(emoji)
      if word not in word_id_pair:
        match = compile.match(word)
        if match:
          grabbedWord = match.group(1)
          if grabbedWord in word_id_pair:
            wordInt.append(word_id_pair[grabbedWord])
          else:
            randomValForWord = random.random()
            word_id_pair[word] = randomValForWord
            wordInt.append(randomValForWord)
      else:
        wordInt.append(word_id_pair[word])
  c_intTweets.append(wordInt)

In [87]:
# Review lengths across test and training whole datasets
print("Maximum review length: {}".format(len(max((c_intTweets), key=len))))
print("Minimum review length: {}".format(len(min((c_intTweets), key=len))))
result = [len(x) for x in c_intTweets]
print("Mean review length: {}".format(np.mean(result)))

#need to provide uniform length
#since mean is 13 and max is 30 we should raise the average to the mid point (21)
review_length = 21

# Padding / truncated our reviews
c_intTweets_padded = sequence.pad_sequences(c_intTweets, maxlen = review_length)

# split the tweet using the same randomseed
# a 80% training, 20 test
c_tweets_train, c_tweets_test, c_y_train, c_y_test = train_test_split(c_intTweets_padded, y, train_size=0.8, random_state=randomSeed)

Maximum review length: 30
Minimum review length: 2
Mean review length: 13.743732590529248


In [88]:
# Train the LSTM on the training data
history = model.fit(

    # Training data : features (review) and classes (positive or negative)
    c_tweets_train, c_y_train,
                    
    # Number of samples to work through before updating the 
    # internal model parameters via back propagation. The 
    # higher the batch, the more memory you need.
    batch_size=128, 

    # An epoch is an iteration over the entire training data.
    epochs=3, 
    
    # The model will set apart his fraction of the training 
    # data, will not train on it, and will evaluate the loss
    # and any model metrics on this data at the end of 
    # each epoch.
    validation_split=0.2,
    
    verbose=1
) 

Epoch 1/3
2/2 [==============================] - 2s 539ms/step - loss: 0.6748 - accuracy: 0.8515 - val_loss: 0.6885 - val_accuracy: 0.5690
Epoch 2/3
2/2 [==============================] - 0s 45ms/step - loss: 0.6729 - accuracy: 0.8908 - val_loss: 0.6875 - val_accuracy: 0.5690
Epoch 3/3
2/2 [==============================] - 0s 42ms/step - loss: 0.6693 - accuracy: 0.8996 - val_loss: 0.6864 - val_accuracy: 0.5862


In [89]:
# Get Model Predictions for test data
from sklearn.metrics import classification_report
predicted_classes = model.predict_classes(c_tweets_test)
classRep = (classification_report(c_y_test, predicted_classes, target_names=class_names))
reportArray = (classRep.split())
precision = (float(reportArray[19])*100.0)
recall = (float(reportArray[20])*100.0)
fOne = (float(reportArray[21])*100.0)
no_at_text_scores = ["No @ text", precision, recall, fOne]

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [90]:
data = [nine_one_split, no_stop_words_scores, no_at_text_scores]
columns = ["Name", "Precision", "Recall", "F1 Measure"]
scores = pd.DataFrame(data=data,columns=columns)
print("Scores")
print(tabulate(scores, headers='keys', tablefmt='fancy_grid'))

Scores
╒════╤═══════════════╤═════════════╤══════════╤══════════════╕
│    │ Name          │   Precision │   Recall │   F1 Measure │
╞════╪═══════════════╪═════════════╪══════════╪══════════════╡
│  0 │ 90/10 Split   │          21 │       50 │           29 │
├────┼───────────────┼─────────────┼──────────┼──────────────┤
│  1 │ No Stop Words │          75 │       63 │           54 │
├────┼───────────────┼─────────────┼──────────┼──────────────┤
│  2 │ No @ text     │          76 │       67 │           59 │
╘════╧═══════════════╧═════════════╧══════════╧══════════════╛


**ANSWER:**



The data has been mostly positively. For the first, splitting the data as 90/10 did not impact the scores much and that could do with pooly tokenized text and noise. For the second and third, the scores drastically and both have to do with removing noise from the tweets, removing the stop words or @texts removed noise that was negatively impacting the lsmt algorithm.